# 03 - Entrenamiento del Modelo (Fine-Tuning de BERT)

**Materia:** Redes Neuronales Profundas — UTN FRM

**Objetivo:** Realizar el fine-tuning de `BertForSequenceClassification` sobre el dataset de Steam Reviews para clasificación binaria de sentimiento.

---

## ¿Qué es el Fine-Tuning?

El fine-tuning es tomar un modelo preentrenado (que aprendió representaciones generales del lenguaje) y adaptarlo a una tarea específica.

**BERT** fue preentrenado en dos tareas:
1. **Masked Language Model (MLM):** Predecir palabras enmascaradas.
2. **Next Sentence Prediction (NSP):** Predecir si dos oraciones son consecutivas.

Durante el fine-tuning:
- Se agrega una **capa de clasificación** lineal encima de BERT (toma el embedding de `[CLS]` y produce 2 salidas).
- Se entrenan **todos los parámetros** con un learning rate muy bajo (2e-5) para no destruir el conocimiento preentrenado.
- Se usa un **scheduler lineal con warmup** para suavizar el entrenamiento.

### Hiperparámetros Recomendados por los Autores de BERT

| Hiperparámetro | Valor | Justificación |
|---|---|---|
| Batch size | 16 o 32 | Sugeridos en el paper original |
| Learning rate | 2e-5, 3e-5, 5e-5 | Tasas bajas para preservar features |
| Épocas | 2 a 4 | Pocas épocas porque el modelo ya sabe mucho |
| Optimizer | AdamW | Adam con weight decay correcto |
| Max seq length | 128 o 512 | Depende de la longitud del texto |

## 1. Importación de Librerías

In [ ]:
import os
import time
import datetime
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

## 2. Configuración

In [ ]:
TENSORS_DIR = "../data/tensors/"
MODEL_SAVE_DIR = "../data/model_save/"
MODEL_NAME = 'bert-base-uncased'
BATCH_SIZE = 32          # Recomendado por BERT: 16 o 32
EPOCHS = 3               # Entre 2 y 4 para fine-tuning
LEARNING_RATE = 2e-5     # Tasa baja para no destruir features preentrenadas
EPSILON = 1e-8           # Epsilon para AdamW
SEED = 42

## 3. Configuración del Dispositivo (GPU)

Verificamos la GPU. Usamos una **RTX 5090 con 32 GB de VRAM**, suficiente para batch_size=32 y secuencias de 128 tokens.

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")
else:
    device = torch.device("cpu")
    print("GPU no disponible, usando CPU.")

# Semilla para reproducibilidad
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

## 4. Carga de los Datasets Tokenizados

In [ ]:
train_dataset = torch.load(os.path.join(TENSORS_DIR, "train_dataset.pt"), weights_only=False)
val_dataset = torch.load(os.path.join(TENSORS_DIR, "val_dataset.pt"), weights_only=False)
print(f"Train: {len(train_dataset):,} muestras")
print(f"Val:   {len(val_dataset):,} muestras")

## 5. Creación de DataLoaders

- **Entrenamiento:** `RandomSampler` — mezcla aleatoriamente cada época.
- **Validación:** `SequentialSampler` — recorre secuencialmente.

In [ ]:
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=BATCH_SIZE)

print(f"Batches de entrenamiento: {len(train_dataloader)}")
print(f"Batches de validación:    {len(validation_dataloader)}")

## 6. Carga del Modelo BERT

Usamos `BertForSequenceClassification`: BERT base + capa lineal de clasificación.

```
Input → BERT Encoder (12 capas transformer) → [CLS] embedding → Linear(768, 2) → Salida
```

In [ ]:
print(f"Cargando modelo {MODEL_NAME}...")
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2, output_attentions=False, output_hidden_states=False,
)
model.to(device)

params = list(model.named_parameters())
print(f"\nEl modelo tiene {len(params)} grupos de parámetros.")
print(f"\n==== Capa de Embedding ====")
for p in params[0:5]:
    print(f"  {p[0]:<55} {str(tuple(p[1].size())):>12}")
print(f"\n==== Primer Transformer ====")
for p in params[5:21]:
    print(f"  {p[0]:<55} {str(tuple(p[1].size())):>12}")
print(f"\n==== Capa de Salida (Clasificación) ====")
for p in params[-4:]:
    print(f"  {p[0]:<55} {str(tuple(p[1].size())):>12}")

## 7. Optimizer y Scheduler

- **AdamW:** Adam con weight decay correcto. lr=2e-5 como recomiendan los autores de BERT.
- **Linear Schedule con Warmup:** Reduce el learning rate linealmente desde el valor inicial hasta 0. Estabiliza el inicio del entrenamiento.

In [ ]:
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, eps=EPSILON)

total_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

print(f"Total de pasos: {total_steps:,}")
print(f"Learning rate: {LEARNING_RATE}")

## 8. Funciones Auxiliares

In [ ]:
def flat_accuracy(preds, labels):
    """Calcula accuracy comparando predicciones vs labels."""
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    """Formatea segundos a hh:mm:ss."""
    return str(datetime.timedelta(seconds=int(round(elapsed))))

## 9. Bucle de Entrenamiento y Validación

Sigue el patrón del notebook de clase (`BERT_Fine_Tuning.ipynb`):

**Por cada época:**
1. **Entrenamiento (`model.train()`):** Forward pass → loss → backward → gradient clipping → optimizer step → scheduler step.
2. **Validación (`model.eval()`):** Forward pass sin gradientes → calcular loss y accuracy.

In [ ]:
training_stats = []
total_t0 = time.time()

for epoch_i in range(EPOCHS):
    print(f'\n{"="*40}')
    print(f'  Época {epoch_i + 1} / {EPOCHS}')
    print(f'{"="*40}')

    # ---- ENTRENAMIENTO ----
    print("\nEntrenando...")
    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and step != 0:
            print(f'  Lote {step:>5,} de {len(train_dataloader):>5,}. Tiempo: {format_time(time.time() - t0)}')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()
        result = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,
                       labels=b_labels, return_dict=True)

        loss = result.loss
        total_train_loss += loss.item()
        loss.backward()

        # Gradient clipping: evita gradientes explosivos
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    print(f"\n  Pérdida promedio: {avg_train_loss:.4f}")
    print(f"  Tiempo: {training_time}")

    # ---- VALIDACIÓN ----
    print("\nValidando...")
    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            result = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,
                           labels=b_labels, return_dict=True)

        total_eval_loss += result.loss.item()
        logits = result.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)

    print(f"  Accuracy: {avg_val_accuracy:.4f}")
    print(f"  Pérdida: {avg_val_loss:.4f}")
    print(f"  Tiempo: {validation_time}")

    training_stats.append({
        'epoch': epoch_i + 1, 'Training Loss': avg_train_loss,
        'Valid. Loss': avg_val_loss, 'Valid. Accur.': avg_val_accuracy,
        'Training Time': training_time, 'Validation Time': validation_time
    })

print(f"\nEntrenamiento completo! Tiempo total: {format_time(time.time() - total_t0)}")

## 10. Resultados del Entrenamiento

In [ ]:
df_stats = pd.DataFrame(data=training_stats).set_index('epoch')
print(df_stats.to_string())

## 11. Guardado del Modelo

Usamos `save_pretrained()` de HuggingFace que guarda los pesos (`model.safetensors`), configuración (`config.json`) y tokenizer.

In [ ]:
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(MODEL_SAVE_DIR)
BertTokenizer.from_pretrained(MODEL_NAME).save_pretrained(MODEL_SAVE_DIR)
print(f"Modelo y tokenizer guardados en: {MODEL_SAVE_DIR}")

df_stats.to_csv(os.path.join(MODEL_SAVE_DIR, "training_stats.csv"))
print("Estadísticas guardadas.")

## Resumen

**Resultados obtenidos:**

| Época | Training Loss | Valid. Loss | Valid. Accuracy |
|---|---|---|---|
| 1 | 0.300 | 0.234 | 90.7% |
| 2 | 0.177 | 0.232 | 91.3% |
| 3 | 0.107 | 0.299 | 91.1% |

La mejor accuracy de validación se obtiene en la época 2 (91.3%). La validation loss aumenta en la época 3, indicando un inicio de overfitting.

**Siguiente paso:** Evaluar en el conjunto de test.